In [1]:
!pip install optimum[openvino]
!pip install gradio diffusers optimum torchaudio moviepy requests serpapi
!pip install google-search-results transformers accelerate yt_dlp

import gradio as gr
import torch
import torchaudio
import os
import warnings
from moviepy.editor import VideoFileClip
from transformers import pipeline, AutoProcessor, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from optimum.intel.openvino import OVModelForSpeechSeq2Seq, OVModelForSeq2SeqLM, OVModelForCausalLM
from io import BytesIO
from serpapi import GoogleSearch
import numpy as np
import gc
import time
import requests
from PIL import Image
import re
import yt_dlp
import tempfile
from urllib.parse import urlparse, parse_qs
import random
from collections import Counter
import json

warnings.filterwarnings("ignore", category=UserWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:0

  if event.key is 'enter':

Multiple distributions found for package optimum. Picked distribution: optimum-intel


In [2]:
UPLOAD_FOLDER = "uploads"
MODEL_CACHE = "model_cache"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(MODEL_CACHE, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Using device: {device}")

# Global model variables
whisper_model = None
whisper_processor = None
notes_model = None
notes_tokenizer = None
chatbot_model = None
chatbot_tokenizer = None

def load_whisper_model():
    """Load Whisper model with OpenVINO optimization"""
    global whisper_model, whisper_processor
    if whisper_model is None:
        print("🔄 Loading Whisper Small model...")
        whisper_model_path = os.path.join(MODEL_CACHE, "whisper-small-ov")

        if not os.path.exists(whisper_model_path):
            print("🔄 Converting Whisper to OpenVINO...")
            whisper_model = OVModelForSpeechSeq2Seq.from_pretrained(
                "openai/whisper-small",
                export=True
            )
            whisper_model.save_pretrained(whisper_model_path)
        else:
            whisper_model = OVModelForSpeechSeq2Seq.from_pretrained(whisper_model_path)

        whisper_processor = AutoProcessor.from_pretrained("openai/whisper-small")
        print("✅ Whisper model loaded!")

def load_notes_model():
    """Load enhanced notes generation model with OpenVINO optimization"""
    global notes_model, notes_tokenizer
    if notes_model is None:
        print("🔄 Loading Qwen2.5-0.5B for enhanced notes generation...")
        notes_model_path = os.path.join(MODEL_CACHE, "qwen2.5-0.5b-notes-ov")

        try:
            if not os.path.exists(notes_model_path):
                print("🔄 Converting Qwen2.5-0.5B to OpenVINO...")
                notes_model = OVModelForCausalLM.from_pretrained(
                    "Qwen/Qwen2.5-0.5B-Instruct",
                    export=True,
                    trust_remote_code=True
                )
                notes_model.save_pretrained(notes_model_path)
            else:
                notes_model = OVModelForCausalLM.from_pretrained(
                    notes_model_path,
                    trust_remote_code=True
                )

            notes_tokenizer = AutoTokenizer.from_pretrained(
                "Qwen/Qwen2.5-0.5B-Instruct",
                trust_remote_code=True
            )
            if notes_tokenizer.pad_token is None:
                notes_tokenizer.pad_token = notes_tokenizer.eos_token

            print("✅ Enhanced Qwen2.5-0.5B model loaded successfully!")
        except Exception as e:
            print(f"⚠️ Qwen2.5 model loading failed: {e}")
            notes_model = None
            notes_tokenizer = None

def cleanup_memory():
    """Enhanced memory cleanup function"""
    try:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        import gc
        gc.collect()

        if 'torch' in globals():
            for obj_name in list(globals().keys()):
                obj = globals()[obj_name]
                if hasattr(obj, 'cpu'):
                    try:
                        obj.cpu()
                        del obj
                    except:
                        pass
    except Exception as e:
        print(f"Memory cleanup warning: {e}")

🔧 Using device: cuda


In [3]:
def load_chatbot_model():
    """Load a lightweight chatbot model for Q&A"""
    global chatbot_model, chatbot_tokenizer
    if chatbot_model is None:
        print("🔄 Loading chatbot model...")
        chatbot_model_path = os.path.join(MODEL_CACHE, "chatbot-model-ov")
        try:
            if not os.path.exists(chatbot_model_path):
                print("🔄 Converting chatbot model to OpenVINO...")
                chatbot_model = OVModelForCausalLM.from_pretrained(
                    "Qwen/Qwen2.5-0.5B-Instruct",
                    export=True,
                    trust_remote_code=True
                )
                chatbot_model.save_pretrained(chatbot_model_path)
            else:
                chatbot_model = OVModelForCausalLM.from_pretrained(
                    chatbot_model_path,
                    trust_remote_code=True
                )

            chatbot_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", trust_remote_code=True)

            if chatbot_tokenizer.pad_token is None:
                chatbot_tokenizer.pad_token = chatbot_tokenizer.eos_token

            print("✅ Chatbot model loaded successfully!")
        except Exception as e:
            print(f"⚠️ Chatbot model loading failed: {e}")
            chatbot_model = None
            chatbot_tokenizer = None

def generate_chatbot_response(message, history):
    """Generate chatbot response using the loaded model"""
    global chat_history

    if not message.strip():
        return history + [("", "Please ask me a question about your studies!")]

    load_chatbot_model()

    if chatbot_model is None or chatbot_tokenizer is None:
        return history + [(message, "Sorry, I'm having trouble loading the chatbot model. Please try again later.")]

    try:
        context = ""
        if len(history) > 0:
            recent_history = history[-2:]
            for user_msg, bot_msg in recent_history:
                if user_msg and bot_msg:
                    context += f"Human: {user_msg}\nAssistant: {bot_msg}\n"

        system_prompt = """You are a knowledgeable and friendly AI teaching assistant. Your goal is to help students learn effectively by:

- Providing complete, well-structured explanations
- Using clear, simple language appropriate for students
- Including relevant examples when helpful
- Breaking down complex concepts step by step
- Always finishing your thoughts completely
- Being encouraging and supportive

Always provide complete answers and end with proper conclusions. Make sure every response is a complete thought that fully addresses the student's question."""

        full_prompt = f"""<|im_start|>system
{system_prompt}
<|im_end|>
"""

        if context:
            full_prompt += f"{context}"

        full_prompt += f"""<|im_start|>user
{message}
<|im_end|>
<|im_start|>assistant
"""

        inputs = chatbot_tokenizer(
            full_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=900,
            padding=True
        )

        with torch.no_grad():
            outputs = chatbot_model.generate(
                **inputs,
                max_new_tokens=1000,
                min_new_tokens=20,
                temperature=0.6,
                do_sample=True,
                top_p=0.85,
                top_k=40,
                pad_token_id=chatbot_tokenizer.pad_token_id,
                eos_token_id=chatbot_tokenizer.eos_token_id,
                early_stopping=False,
                no_repeat_ngram_size=3,
                repetition_penalty=1.15,
                length_penalty=1.0,
                num_return_sequences=1
            )

        full_response = chatbot_tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "<|im_start|>assistant" in full_response:
            bot_response = full_response.split("<|im_start|>assistant")[-1].strip()
        else:
            prompt_length = len(chatbot_tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True))
            bot_response = full_response[prompt_length:].strip()

        bot_response = enhance_chatbot_response(bot_response)

        if not bot_response or len(bot_response.split()) < 5:
            bot_response = generate_fallback_response(message)

        del inputs, outputs
        cleanup_memory()

        new_history = history + [(message, bot_response)]
        return new_history

    except Exception as e:
        error_response = "I apologize, but I encountered an error while processing your question. Please try asking again or rephrase your question, and I'll do my best to help!"
        return history + [(message, error_response)]

def enhance_chatbot_response(response):
    """Enhanced cleaning and completion of chatbot response"""
    try:
        response = re.sub(r'<\|.*?\|>', '', response)
        response = re.sub(r'<.*?>', '', response)
        response = re.sub(r'\|.*?\|', '', response)

        response = re.sub(r'^(Assistant:|AI:|Bot:|Response:)\s*', '', response, flags=re.IGNORECASE)

        response = re.sub(r'\s+', ' ', response).strip()

        sentences = re.split(r'(?<=[.!?])\s+', response)
        cleaned_sentences = []

        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 5:
                if sentence and sentence[0].islower():
                    sentence = sentence[0].upper() + sentence[1:]
                cleaned_sentences.append(sentence)

        cleaned_response = ' '.join(cleaned_sentences)

        if cleaned_response and not cleaned_response.endswith(('.', '!', '?', ':')):
            if cleaned_response.endswith(',') or cleaned_response.endswith(' and') or cleaned_response.endswith(' or'):
                cleaned_response = cleaned_response.rstrip(', ') + '.'
            else:
                cleaned_response += '.'

        sentences = cleaned_response.split('. ')
        unique_sentences = []
        for sentence in sentences:
            if sentence not in unique_sentences and len(sentence.strip()) > 3:
                unique_sentences.append(sentence)

        final_response = '. '.join(unique_sentences)

        if not final_response.endswith(('.', '!', '?')):
            final_response += '.'

        return final_response if final_response and len(final_response.split()) >= 3 else response

    except Exception as e:
        return response

def generate_fallback_response(question):
    """Generate a fallback response when the model fails"""
    question_lower = question.lower()

    if any(word in question_lower for word in ['math', 'mathematics', 'algebra', 'calculus', 'geometry']):
        return "I'd be happy to help with your math question! Mathematics involves logical problem-solving and step-by-step thinking. Could you please provide more specific details about the concept or problem you're working on? I can then break it down into manageable steps."

    elif any(word in question_lower for word in ['science', 'physics', 'chemistry', 'biology']):
        return "Science is all about understanding how the world works! I'm here to help explain scientific concepts in simple terms. Could you please specify which area of science you're asking about, or rephrase your question? I'll do my best to provide a clear explanation."

    elif any(word in question_lower for word in ['history', 'historical', 'past', 'timeline']):
        return "History helps us understand the past and learn from it! I can help explain historical events, their causes and effects, and their significance. Please provide more details about the specific historical topic or period you're interested in."

    elif any(word in question_lower for word in ['english', 'literature', 'writing', 'grammar']):
        return "Language and literature are powerful tools for communication and expression! I can help with grammar, writing techniques, literary analysis, and more. What specific aspect of English or literature would you like help with?"

    else:
        return "I understand you have a question about your studies. I'm here to help explain concepts, provide examples, and support your learning journey. Could you please rephrase your question or provide a bit more context? This will help me give you a more detailed and helpful answer."

def clear_chat_history():
    """Clear the chat history"""
    return []

In [4]:
def is_youtube_url(url):
    """Check if the provided URL is a valid YouTube URL"""
    if not url:
        return False

    youtube_patterns = [
        r'(?:https?://)?(?:www\.)?youtube\.com/watch\?v=([a-zA-Z0-9_-]+)',
        r'(?:https?://)?(?:www\.)?youtu\.be/([a-zA-Z0-9_-]+)',
        r'(?:https?://)?(?:www\.)?youtube\.com/embed/([a-zA-Z0-9_-]+)',
        r'(?:https?://)?(?:www\.)?youtube\.com/v/([a-zA-Z0-9_-]+)'
    ]

    for pattern in youtube_patterns:
        if re.search(pattern, url):
            return True
    return False

In [5]:
def download_youtube_video(url, progress_callback=None, max_retries=3):
    """
    Download YouTube video with robust error handling and retry logic
    """
    try:
        if not is_youtube_url(url):
            return None, "❌ Invalid YouTube URL provided"
        temp_dir = tempfile.mkdtemp()

        ydl_opts = {

            'format': 'worst[height>=360][ext=mp4]/worst[ext=mp4]/worst',
            'outtmpl': os.path.join(temp_dir, '%(title).50s.%(ext)s'),
            'writesubtitles': False,
            'writeautomaticsub': False,
            'extract_flat': False,
            'no_warnings': True,
            'quiet': True,
            'socket_timeout': 30,
            'retries': 3,
            'fragment_retries': 3,
            'http_chunk_size': 1048576,
            'http_headers': {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
        }


        if progress_callback:
            def progress_hook(d):
                try:
                    if d['status'] == 'downloading':
                        if '_percent_str' in d:
                            percent_str = d['_percent_str'].replace('%', '')
                            percent = float(percent_str)
                            progress_callback(percent / 100, f"📥 Downloading: {percent:.1f}%")
                        else:
                            progress_callback(0.5, "📥 Downloading YouTube video...")
                    elif d['status'] == 'finished':
                        progress_callback(1.0, "✅ Download complete!")
                except:
                    progress_callback(0.5, "📥 Downloading...")

            ydl_opts['progress_hooks'] = [progress_hook]

        for attempt in range(max_retries):
            try:
                if progress_callback:
                    progress_callback(0.05, f"🔄 Attempt {attempt + 1}/{max_retries}...")

                with yt_dlp.YoutubeDL(ydl_opts) as ydl:

                    if progress_callback:
                        progress_callback(0.1, "🔍 Extracting video info...")

                    info = ydl.extract_info(url, download=False)
                    if not info:
                        return None, "❌ Could not extract video information"

                    video_title = info.get('title', 'Unknown')[:50]
                    duration = info.get('duration', 0)

                    if duration and duration > 7200:
                        return None, f"❌ Video too long ({duration//60} minutes). Please use videos under 2 hours."

                    if progress_callback:
                        progress_callback(0.15, f"📺 Found: {video_title}...")

                    ydl.download([url])

                    downloaded_files = [f for f in os.listdir(temp_dir)
                                     if os.path.isfile(os.path.join(temp_dir, f)) and not f.startswith('.')]

                    if downloaded_files:
                        video_path = os.path.join(temp_dir, downloaded_files[0])

                        if os.path.exists(video_path) and os.path.getsize(video_path) > 0:
                            return video_path, f"✅ Downloaded: {video_title}"
                        else:
                            raise Exception("Downloaded file is empty or corrupted")
                    else:
                        raise Exception("No files found after download")

            except yt_dlp.utils.DownloadError as e:
                error_msg = str(e)
                if "HTTP Error 429" in error_msg:
                    if attempt < max_retries - 1:
                        if progress_callback:
                            progress_callback(0.3, f"⏳ Rate limited, waiting before retry {attempt + 2}...")
                        time.sleep(5)
                        continue
                    else:
                        return None, "❌ YouTube rate limit exceeded. Please try again later."
                elif "timeout" in error_msg.lower():
                    if attempt < max_retries - 1:
                        if progress_callback:
                            progress_callback(0.3, f"⏳ Connection timeout, retrying {attempt + 2}...")
                        time.sleep(2)
                        continue
                    else:
                        return None, "❌ Connection timeout. Please check your internet connection and try again."
                else:
                    if attempt < max_retries - 1:
                        if progress_callback:
                            progress_callback(0.3, f"⏳ Download error, retrying {attempt + 2}...")
                        time.sleep(2)
                        continue
                    else:
                        return None, f"❌ Download failed after {max_retries} attempts: {error_msg}"

            except Exception as e:
                if attempt < max_retries - 1:
                    if progress_callback:
                        progress_callback(0.3, f"⏳ Error occurred, retrying {attempt + 2}...")
                    time.sleep(2)
                    continue
                else:
                    return None, f"❌ Download failed after {max_retries} attempts: {str(e)}"

        return None, f"❌ Download failed after {max_retries} attempts"

    except Exception as e:
        return None, f"❌ YouTube download error: {str(e)}"

In [6]:
def download_youtube_audio_only(url, progress_callback=None):
    """
    Download only audio from YouTube video (more reliable for transcription)
    """
    try:
        if not is_youtube_url(url):
            return None, "❌ Invalid YouTube URL provided"

        temp_dir = tempfile.mkdtemp()

        ydl_opts = {
            'format': 'bestaudio[ext=m4a]/bestaudio/best',
            'outtmpl': os.path.join(temp_dir, '%(title).50s.%(ext)s'),
            'cookiesfrombrowser': ('chrome',),
            'writesubtitles': False,
            'writeautomaticsub': False,
            'extract_flat': False,
            'no_warnings': True,
            'quiet': True,
            'socket_timeout': 30,
            'retries': 5,
            'fragment_retries': 5,
            'sleep_interval': random.randint(1, 3),
            'prefer_ffmpeg': True,
            'keepvideo': False,
            'postprocessors': [{
              'key': 'FFmpegExtractAudio',
              'preferredcodec': 'mp3',
              'preferredquality': '192',
            }],
            'http_headers': {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
        }

        if progress_callback:
            def progress_hook(d):
                try:
                    if d['status'] == 'downloading':
                        if '_percent_str' in d:
                            percent = float(d['_percent_str'].replace('%', ''))
                            progress_callback(percent / 100, f"🎵 Downloading audio: {percent:.1f}%")
                        else:
                            progress_callback(0.5, "🎵 Downloading audio...")
                    elif d['status'] == 'finished':
                        progress_callback(1.0, "✅ Audio download complete!")
                except:
                    progress_callback(0.5, "🎵 Downloading...")

            ydl_opts['progress_hooks'] = [progress_hook]

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            if progress_callback:
                progress_callback(0.1, "🔍 Extracting video info...")

            info = ydl.extract_info(url, download=False)
            if not info:
                return None, "❌ Could not extract video information"

            video_title = info.get('title', 'Unknown')[:50]
            duration = info.get('duration', 0)

            if duration and duration > 7200:
                return None, f"❌ Video too long ({duration//60} minutes). Please use videos under 2 hours."

            if progress_callback:
                progress_callback(0.15, f"🎵 Downloading audio for: {video_title}...")

            ydl.download([url])

            downloaded_files = [f for f in os.listdir(temp_dir)
                             if os.path.isfile(os.path.join(temp_dir, f)) and not f.startswith('.')]

            if downloaded_files:
                audio_path = os.path.join(temp_dir, downloaded_files[0])
                if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                    return audio_path, f"✅ Downloaded audio: {video_title}"
                else:
                    return None, "❌ Downloaded file is empty"
            else:
                return None, "❌ No audio file found after download"

    except Exception as e:
        return None, f"❌ Audio download error: {str(e)}"

In [7]:
def transcribe_audio_from_video(video_file, progress=gr.Progress()):
    if not video_file:
        return "❌ No video uploaded."

    load_whisper_model()

    try:
        progress(0, desc="Extracting audio...")
        video = VideoFileClip(video_file)
        if not video.audio:
            video.close()
            return "❌ No audio found in video."

        audio_path = os.path.join(UPLOAD_FOLDER, "audio.wav")
        video.audio.write_audiofile(audio_path, logger=None)
        video.close()

        del video
        cleanup_memory()

        waveform, sample_rate = torchaudio.load(audio_path)
        if sample_rate != 16000:
            resample = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resample(waveform)
            sample_rate = 16000

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        progress(0.2, desc="Processing audio chunks...")

        chunk_length_sec = 20
        chunk_size = chunk_length_sec * sample_rate
        stride = int(sample_rate * 4)
        transcripts = []

        total_length = waveform.size(1)
        batch_size = chunk_size * 3

        for batch_start in range(0, total_length, batch_size):
            batch_end = min(batch_start + batch_size, total_length)
            batch_waveform = waveform[:, batch_start:batch_end]

            for start in range(0, batch_waveform.size(1), chunk_size - stride):
                end = min(start + chunk_size, batch_waveform.size(1))
                chunk = batch_waveform[:, start:end]

                if chunk.size(1) < 8000:
                    continue

                try:
                    inputs = whisper_processor(chunk.squeeze(0), sampling_rate=16000, return_tensors="pt")

                    with torch.no_grad():
                        generated_ids = whisper_model.generate(inputs["input_features"])

                    text = whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
                    if text and text.strip():
                        transcripts.append(text.strip())

                    del inputs, generated_ids

                except Exception as e:
                    print(f"❌ Chunk processing failed: {e}")
                    continue

            del batch_waveform
            cleanup_memory()

            progress_val = min(0.9, 0.2 + (batch_end / total_length) * 0.7)
            progress(progress_val, desc=f"Processing audio... {int((batch_end/total_length)*100)}%")

        if os.path.exists(audio_path):
            os.remove(audio_path)

        del waveform
        cleanup_memory()

        progress(1.0, desc="Transcription complete!")

        if not transcripts:
            return "❌ No speech detected in video. Try a video with clearer audio."

        final_transcript = " ".join(transcripts)
        return final_transcript if final_transcript.strip() else "❌ No clear speech detected."

    except Exception as e:
        cleanup_memory()
        return f"❌ Error processing video: {str(e)}"

In [8]:
def detect_subject_area(transcript):
    """Detect the subject area of the transcript"""
    text_lower = transcript.lower()

    subject_keywords = {
        'computer_science': ['algorithm', 'programming', 'computer', 'software', 'code', 'data structure', 'database', 'network', 'ai', 'machine learning'],
        'mathematics': ['equation', 'theorem', 'proof', 'mathematical', 'formula', 'calculus', 'algebra', 'geometry', 'statistics'],
        'physics': ['force', 'energy', 'particle', 'quantum', 'physics', 'motion', 'wave', 'electromagnetic', 'thermodynamics'],
        'chemistry': ['molecule', 'atom', 'chemical', 'reaction', 'compound', 'element', 'bond', 'solution', 'acid', 'base'],
        'biology': ['cell', 'organism', 'genetic', 'biology', 'species', 'evolution', 'dna', 'protein', 'ecosystem'],
        'business': ['market', 'business', 'management', 'strategy', 'finance', 'economics', 'profit', 'customer', 'sales'],
        'history': ['historical', 'century', 'war', 'empire', 'civilization', 'culture', 'society', 'revolution', 'ancient'],
        'literature': ['author', 'novel', 'poem', 'literary', 'character', 'theme', 'narrative', 'metaphor', 'symbolism']
    }

    scores = {}
    for subject, keywords in subject_keywords.items():
        score = sum(1 for keyword in keywords if keyword in text_lower)
        scores[subject] = score

    # Return the subject with highest score, or 'general' if no clear match
    if max(scores.values()) > 2:
        return max(scores, key=scores.get)
    return 'general'

def split_transcript_intelligently(transcript, max_length):
    """Split transcript at sentence boundaries to preserve context"""
    if len(transcript) <= max_length:
        return [transcript]

    chunks = []
    sentences = re.split(r'[.!?]+', transcript)
    current_chunk = ""

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        # Check if adding this sentence would exceed the limit
        if len(current_chunk) + len(sentence) + 2 <= max_length:
            current_chunk = current_chunk + ". " + sentence if current_chunk else sentence
        else:
            if current_chunk:
                chunks.append(current_chunk + ".")
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk + ".")

    return chunks

def generate_enhanced_chunk_notes(chunk, subject_area, is_first_chunk):
    """Generate enhanced notes for a single chunk with subject-specific prompting"""

    # Subject-specific prompting
    subject_prompts = {
        'computer_science': "Focus on algorithms, data structures, programming concepts, and technical implementations.",
        'mathematics': "Emphasize formulas, theorems, proofs, and mathematical relationships.",
        'physics': "Highlight physical laws, equations, phenomena, and their applications.",
        'chemistry': "Focus on chemical reactions, molecular structures, and chemical principles.",
        'biology': "Emphasize biological processes, organisms, and life science concepts.",
        'business': "Focus on business strategies, market concepts, and management principles.",
        'history': "Emphasize historical events, dates, causes, and consequences.",
        'literature': "Focus on literary devices, themes, characters, and narrative techniques.",
        'general': "Focus on key concepts, important information, and main ideas."
    }

    subject_instruction = subject_prompts.get(subject_area, subject_prompts['general'])

    prompt = f"""<|im_start|>system
You are an expert note-taking assistant specializing in creating comprehensive, well-structured academic notes. Create detailed notes from lecture content with clear organization and formatting.

{subject_instruction}

Format your response with these sections:
- KEY CONCEPTS: Main ideas and important principles
- DETAILED EXPLANATIONS: In-depth explanations of complex topics
- IMPORTANT TERMS: Definitions and terminology
- EXAMPLES: Practical examples and applications
- SUMMARY: Brief overview of main points

Use clear, academic language and ensure completeness. Always finish your sentences properly and avoid cutting off thoughts mid-sentence.
<|im_end|>

<|im_start|>user
Create comprehensive structured notes from this lecture content:

{chunk}

Make the notes detailed, well-organized, and academically rigorous. Include explanations that would help a student understand the material thoroughly. Ensure all sentences are complete and properly concluded.
<|im_end|>

<|im_start|>assistant
KEY CONCEPTS:
"""

    try:
        inputs = notes_tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1600,
            padding=False,
            return_attention_mask=True
        )

        with torch.no_grad():
            outputs = notes_model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=800,
                min_new_tokens=50,  # Ensure minimum length
                temperature=0.2,
                do_sample=True,
                top_p=0.8,
                top_k=25,
                pad_token_id=notes_tokenizer.pad_token_id,
                eos_token_id=notes_tokenizer.eos_token_id,
                early_stopping=False,  # Don't stop early
                no_repeat_ngram_size=4,
                repetition_penalty=1.15,
                length_penalty=1.2
            )

        response = notes_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the assistant's response
        if "<|im_start|>assistant" in response:
            notes = response.split("<|im_start|>assistant")[-1].strip()
        else:
            notes = response[len(prompt):].strip()

        # Clean and enhance the notes with sentence completion
        notes = clean_and_enhance_notes_with_completion(notes)

        del inputs, outputs
        return notes if notes and len(notes.strip()) > 50 else ""

    except Exception as e:
        print(f"❌ Enhanced chunk generation error: {str(e)}")
        return ""

def clean_and_enhance_notes_with_completion(notes):
    """Clean and enhance generated notes with better formatting and sentence completion"""
    try:
        # Remove special tokens and artifacts
        notes = re.sub(r'<\|im_start\|>.*?<\|im_end\|>', '', notes, flags=re.DOTALL)
        notes = re.sub(r'<\|.*?\|>', '', notes)

        # Split into lines and process
        lines = notes.split('\n')
        cleaned_lines = []
        current_section = ""

        for line in lines:
            line = line.strip()
            if not line or len(line) < 3:
                continue

            # Identify section headers
            if any(header in line.upper() for header in ['KEY CONCEPTS', 'DETAILED EXPLANATIONS', 'IMPORTANT TERMS', 'EXAMPLES', 'SUMMARY']):
                current_section = line.upper()
                cleaned_lines.append(f"\n{current_section}:")
                continue

            # Clean and format content lines
            if line.startswith(('- ', '• ', '* ')):
                line = line[2:].strip()

            # Fix incomplete sentences at the beginning
            if line and not line[0].isupper() and not line[0].isdigit():
                # Skip lines that start with lowercase (likely cut-off)
                continue

            # Fix incomplete sentences at the end
            if line.endswith(('and', 'the', 'of', 'in', 'to', 'for', 'with', 'by', 'from', 'that', 'which', 'this', 'these', 'those')):
                # Try to complete the sentence or skip if too incomplete
                if len(line.split()) < 5:
                    continue
                else:
                    line = line.rstrip('and the of in to for with by from that which this these those').strip()
                    if line and not line.endswith(('.', '!', '?')):
                        line += '.'

            # Ensure proper capitalization
            if line and line[0].islower():
                line = line[0].upper() + line[1:]

            # Ensure proper sentence ending
            if line and not line.endswith(('.', '!', '?', ':')):
                line += '.'

            # Add bullet point formatting
            if current_section and not line.startswith(('1.', '2.', '3.', '4.', '5.')):
                line = f"• {line}"

            # Remove duplicates and very short lines
            if line not in cleaned_lines and len(line) > 15:
                cleaned_lines.append(line)

        # Final pass to ensure sentence completion
        final_lines = []
        for line in cleaned_lines:
            if line.strip():
                # Check if line ends abruptly
                if line.endswith(',') or (line.count(' ') > 3 and not line.endswith(('.', '!', '?', ':'))):
                    line = line.rstrip(',') + '.'
                final_lines.append(line)

        return '\n'.join(final_lines)

    except Exception as e:
        return notes

def generate_subject_specific_study_tips(subject_area):
    """Generate study tips specific to the subject area"""
    tips = {
        'computer_science': [
            "Practice coding the algorithms and data structures mentioned",
            "Draw diagrams to visualize complex data structures",
            "Implement examples in your preferred programming language",
            "Review time and space complexity analysis"
        ],
        'mathematics': [
            "Work through example problems step by step",
            "Create a formula sheet with key equations",
            "Practice proofs and understand the logical flow",
            "Visualize concepts with graphs and diagrams"
        ],
        'physics': [
            "Understand the physical meaning behind equations",
            "Practice problem-solving with different scenarios",
            "Create concept maps linking related phenomena",
            "Review units and dimensional analysis"
        ],
        'chemistry': [
            "Draw molecular structures and reaction mechanisms",
            "Practice balancing chemical equations",
            "Understand periodic trends and their applications",
            "Create flashcards for important reactions"
        ],
        'biology': [
            "Create detailed diagrams of biological processes",
            "Use mnemonics to remember complex pathways",
            "Connect concepts across different biological levels",
            "Practice with case studies and real examples"
        ],
        'business': [
            "Apply concepts to real-world business scenarios",
            "Create case study analyses",
            "Review current market examples",
            "Practice with financial calculations and models"
        ],
        'history': [
            "Create timelines of important events",
            "Understand cause-and-effect relationships",
            "Connect historical events to modern contexts",
            "Practice essay writing with historical evidence"
        ],
        'literature': [
            "Analyze themes and literary devices in detail",
            "Create character analysis charts",
            "Practice close reading techniques",
            "Connect works to their historical and cultural contexts"
        ],
        'general': [
            "Review and summarize key points regularly",
            "Create concept maps to connect ideas",
            "Practice active recall with the material",
            "Discuss concepts with peers or study groups"
        ]
    }

    return tips.get(subject_area, tips['general'])


def generate_structured_notes(transcript):
    """Generate enhanced structured notes using improved Qwen2.5-0.5B model"""
    if not transcript or len(transcript.strip()) < 100:
        return "❌ Transcript too short to generate meaningful notes."

    # Load model if not already loaded
    if notes_model is None or notes_tokenizer is None:
        load_notes_model()
        if notes_model is None or notes_tokenizer is None:
            return "❌ Failed to load notes generation model."

    try:
        # Analyze transcript to determine subject area
        subject_area = detect_subject_area(transcript)

        # Split transcript intelligently
        max_chunk_length = 1500
        chunks = split_transcript_intelligently(transcript, max_chunk_length)

        all_notes_sections = []

        # Process each chunk with subject-specific prompting
        for i, chunk in enumerate(chunks[:4]):  # Process up to 4 chunks
            try:
                chunk_notes = generate_enhanced_chunk_notes(chunk, subject_area, i == 0)
                if chunk_notes and len(chunk_notes.strip()) > 50:
                    all_notes_sections.append(chunk_notes)
                cleanup_memory()
            except Exception as e:
                print(f"⚠️ Error processing chunk {i}: {e}")
                continue

        if all_notes_sections:
            # Merge and structure all notes
            merged_notes = merge_and_structure_notes(all_notes_sections, subject_area)
            final_notes = format_enhanced_notes(merged_notes, subject_area)
            return final_notes
        else:
            return "❌ Failed to generate notes from the provided transcript."

    except Exception as e:
        print(f"❌ Enhanced notes generation error: {str(e)}")
        cleanup_memory()
        return f"❌ Error generating notes: {str(e)}"

def merge_and_structure_notes(notes_sections, subject_area):
    """Merge multiple note sections into a structured format"""
    merged = {
        'key_concepts': [],
        'detailed_explanations': [],
        'important_terms': [],
        'examples': [],
        'summary': []
    }

    for section in notes_sections:
        parsed = parse_enhanced_sections(section)
        for key, items in parsed.items():
            if key in merged:
                merged[key].extend(items)

    # Remove duplicates and limit items
    for key in merged:
        merged[key] = remove_duplicates_enhanced(merged[key])[:12]

    return merged

def parse_enhanced_sections(notes):
    """Parse notes into enhanced sections"""
    sections = {
        'key_concepts': [],
        'detailed_explanations': [],
        'important_terms': [],
        'examples': [],
        'summary': []
    }

    current_section = 'key_concepts'
    lines = notes.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue

        line_upper = line.upper()

        # Identify section headers
        if 'KEY CONCEPTS' in line_upper:
            current_section = 'key_concepts'
        elif 'DETAILED EXPLANATIONS' in line_upper or 'EXPLANATIONS' in line_upper:
            current_section = 'detailed_explanations'
        elif 'IMPORTANT TERMS' in line_upper or 'TERMS' in line_upper or 'DEFINITIONS' in line_upper:
            current_section = 'important_terms'
        elif 'EXAMPLES' in line_upper or 'APPLICATIONS' in line_upper:
            current_section = 'examples'
        elif 'SUMMARY' in line_upper:
            current_section = 'summary'
        else:
            # Add content to current section
            if line.startswith('•'):
                line = line[1:].strip()
            elif line.startswith(('-', '*')):
                line = line[1:].strip()

            if len(line) > 15 and current_section in sections:
                sections[current_section].append(line)

    return sections

def remove_duplicates_enhanced(items):
    """Remove duplicates while preserving order and quality"""
    seen = set()
    result = []

    for item in items:
        # Create a key for comparison (first 60 characters, lowercased)
        item_key = item.lower().strip()[:60]

        # Skip if too similar to existing items
        is_duplicate = False
        for seen_key in seen:
            if len(set(item_key.split()) & set(seen_key.split())) > len(item_key.split()) * 0.7:
                is_duplicate = True
                break

        if not is_duplicate and item_key not in seen:
            seen.add(item_key)
            result.append(item)

    return result

def format_enhanced_notes(merged_notes, subject_area):
    """Format merged notes with enhanced structure and subject-specific formatting"""
    output = []

    # Header with subject area
    subject_display = subject_area.replace('_', ' ').title()
    output.append(f"COMPREHENSIVE LECTURE NOTES - {subject_display}")
    output.append("=" * 60)
    output.append("")

    # Key Concepts Section
    if merged_notes['key_concepts']:
        output.append("🔑 KEY CONCEPTS:")
        output.append("-" * 40)
        for i, concept in enumerate(merged_notes['key_concepts'], 1):
            output.append(f"{i}. {concept}")
        output.append("")

    # Detailed Explanations Section
    if merged_notes['detailed_explanations']:
        output.append("📚 DETAILED EXPLANATIONS:")
        output.append("-" * 40)
        for i, explanation in enumerate(merged_notes['detailed_explanations'], 1):
            output.append(f"{i}. {explanation}")
        output.append("")

    # Important Terms Section
    if merged_notes['important_terms']:
        output.append("📖 IMPORTANT TERMS & DEFINITIONS:")
        output.append("-" * 40)
        for i, term in enumerate(merged_notes['important_terms'], 1):
            output.append(f"{i}. {term}")
        output.append("")

    # Examples Section
    if merged_notes['examples']:
        output.append("💡 EXAMPLES & APPLICATIONS:")
        output.append("-" * 40)
        for i, example in enumerate(merged_notes['examples'], 1):
            output.append(f"{i}. {example}")
        output.append("")

    # Summary Section
    if merged_notes['summary']:
        output.append("📝 SUMMARY:")
        output.append("-" * 40)
        for i, summary_point in enumerate(merged_notes['summary'], 1):
            output.append(f"{i}. {summary_point}")
        output.append("")

    # Study Tips Section
    output.append("💡 STUDY TIPS:")
    output.append("-" * 40)
    study_tips = generate_subject_specific_study_tips(subject_area)
    for i, tip in enumerate(study_tips, 1):
        output.append(f"{i}. {tip}")
    output.append("")

    return '\n'.join(output)

In [9]:
def search_google_images(query, num_images=2):
    if not query.strip():
        return []
    try:
        search = GoogleSearch({
            "q": query,
            "tbm": "isch",
            "api_key": "cbc9aa754b28b7a45c57feb677147a418d633f661ef678900c64e24bc52c379a"
        })
        results = search.get_dict()
        if "error" in results:
            return []

        images_results = results.get("images_results", [])
        image_urls = []
        for img in images_results[:num_images]:
            original_url = img.get("original")
            if original_url:
                image_urls.append(original_url)
        return image_urls
    except Exception as e:
        print(f"❌ Image search error: {e}")
        return []

In [10]:
def download_image_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            return image.convert('RGB')
        return None
    except Exception as e:
        print(f"❌ Image download failed: {e}")
        return None

In [11]:
def search_web_enhanced(query, num_results=8, include_videos=True):
    if not query.strip():
        return "❌ No search query provided."
    try:
        search_params = {
            "q": query,
            "api_key": "cbc9aa754b28b7a45c57feb677147a418d633f661ef678900c64e24bc52c379a",
            "num": num_results
        }
        search = GoogleSearch(search_params)
        results = search.get_dict()

        if "error" in results:
            return f"❌ Search API error: {results['error']}"

        organic_results = results.get("organic_results", [])
        formatted_results = []

        if organic_results:
            formatted_results.append("🔍 **WEB RESULTS:**\n")
            for i, result in enumerate(organic_results[:num_results], 1):
                title = result.get("title", "No title")
                link = result.get("link", "")
                snippet = result.get("snippet", "No description available")
                result_text = f"{i}. **{title}**\n{snippet}\n🔗 {link}\n"
                formatted_results.append(result_text)

        if include_videos:
            video_search = GoogleSearch({
                "q": query,
                "tbm": "vid",
                "api_key": "cbc9aa754b28b7a45c57feb677147a418d633f661ef678900c64e24bc52c379a",
                "num": 5
            })
            video_results = video_search.get_dict()
            video_results_data = video_results.get("video_results", [])

            if video_results_data:
                formatted_results.append("\n📹 **VIDEO RESULTS:**\n")
                for i, video in enumerate(video_results_data[:5], 1):
                    title = video.get("title", "No title")
                    link = video.get("link", "")
                    duration = video.get("duration", "")
                    channel = video.get("channel", "")
                    video_text = f"{i}. **{title}**"
                    if duration:
                        video_text += f" ({duration})"
                    if channel:
                        video_text += f" - {channel}"
                    video_text += f"\n🎥 {link}\n"
                    formatted_results.append(video_text)

        return "\n".join(formatted_results) if formatted_results else "❌ No results found."
    except Exception as e:
        return f"❌ Search error: {str(e)}"

In [12]:
def extract_keywords_and_concepts(text, max_keywords=8):
    import re
    priority_terms = [
        'theory', 'concept', 'principle', 'method', 'process', 'system', 'model',
        'analysis', 'research', 'study', 'experiment', 'data', 'result', 'conclusion',
        'algorithm', 'function', 'equation', 'formula', 'definition', 'example',
        'application', 'implementation', 'solution', 'problem', 'approach'
    ]

    words = re.findall(r'\b[a-zA-Z]{4,}\b', text.lower())
    word_freq = {}
    for word in words:
        if len(word) >= 4:
            word_freq[word] = word_freq.get(word, 0) + 1

    for word in word_freq:
        if any(term in word for term in priority_terms):
            word_freq[word] *= 2

    top_keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_keywords]
    return [word for word, freq in top_keywords]

In [13]:
def create_search_query(transcript):
    keywords = extract_keywords_and_concepts(transcript[:500])
    if len(keywords) >= 3:
        return " ".join(keywords[:3]) + " tutorial explanation"
    return " ".join(keywords[:2]) + " educational content" if keywords else "educational tutorial"

In [14]:
def process_youtube_or_upload_robust(video_file, youtube_url, num_search_results=8, num_images=2, progress=gr.Progress()):
    """
    Enhanced pipeline with fallback to audio-only download for YouTube
    """
    video_path = None
    temp_file_path = None

    try:
        if youtube_url and youtube_url.strip():
            if not is_youtube_url(youtube_url.strip()):
                return "❌ Invalid YouTube URL format.", "", None, None, ""

            progress(0, desc="🔗 Processing YouTube URL...")

            temp_file_path, download_msg = download_youtube_video(
                youtube_url.strip(),
                progress_callback=lambda p, desc: progress(p * 0.2, desc)
            )

            if not temp_file_path:
                progress(0.05, desc="🎵 Trying audio-only download...")
                temp_file_path, download_msg = download_youtube_audio_only(
                    youtube_url.strip(),
                    progress_callback=lambda p, desc: progress(0.05 + p * 0.25, desc)
                )

            if not temp_file_path:
                return download_msg, "", None, None, ""

            video_path = temp_file_path
            progress(0.3, desc="✅ YouTube content ready for processing!")

        elif video_file:
            video_path = video_file
            progress(0.3, desc="✅ Uploaded video ready for processing!")
        else:
            return "❌ Please either upload a video file or provide a YouTube URL.", "", None, None, ""

        progress(0.35, desc="🎬 Starting transcription...")
        transcript = transcribe_audio_from_video(video_path, progress=lambda p, desc: progress(0.35 + p * 0.25, desc))
        if transcript.startswith("❌"):
            return transcript, "", None, None, ""
        progress(0.6, desc="✅ Transcription complete!")

        progress(0.65, desc="📝 Creating structured notes...")
        structured_notes = generate_structured_notes(transcript)
        progress(0.75, desc="✅ Structured notes created!")

        progress(0.8, desc="🔍 Analyzing content for search...")
        search_query = create_search_query(transcript)
        keywords = extract_keywords_and_concepts(transcript[:500])

        progress(0.85, desc="🌐 Performing web search...")
        search_results = search_web_enhanced(search_query, num_search_results, include_videos=True)
        progress(0.9, desc="✅ Search complete!")

        progress(0.95, desc="🖼️ Searching for relevant images...")
        image_query = " ".join(keywords[:4]) if len(keywords) >= 4 else search_query
        image_urls = search_google_images(image_query, num_images)

        images = []
        for i, url in enumerate(image_urls):
            img = download_image_from_url(url)
            images.append(img)

        image1 = images[0] if len(images) > 0 else None
        image2 = images[1] if len(images) > 1 else None

        progress(1.0, desc="✅ Processing complete!")

        return transcript, structured_notes, image1, image2, search_results

    except Exception as e:
        error_msg = f"❌ Pipeline error: {str(e)}"
        return error_msg, "", None, None, ""

    finally:
        if temp_file_path and os.path.exists(temp_file_path):
            try:
                os.remove(temp_file_path)
                os.rmdir(os.path.dirname(temp_file_path))
            except:
                pass
        cleanup_memory()

In [15]:
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

:root {
    --primary-color-light: #007bff;
    --secondary-color-light: #6c757d;
    --bg-color-light: #ffffff;
    --text-color-light: #000000;
    --border-color-light: #dee2e6;

    --primary-color-dark: #0d6efd;
    --secondary-color-dark: #6c757d;
    --bg-color-dark: #1a1a1a;
    --text-color-dark: #ffffff;
    --border-color-dark: #404040;
}

/* Light mode styles */
.gradio-container {
    background-color: var(--bg-color-light);
    color: var(--text-color-light);
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, sans-serif;
    font-weight: 400;
    line-height: 1.5;
}

/* Dark mode styles */
.dark .gradio-container {
    background-color: var(--bg-color-dark) !important;
    color: var(--text-color-dark) !important;
}

/* Input fields */
.gr-textbox, .gr-textbox textarea, .gr-textbox input,
textarea, input[type="text"] {
    background-color: var(--bg-color-light);
    color: var(--text-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 6px;
    font-family: 'Inter', sans-serif;
    font-size: 14px;
    transition: border-color 0.2s ease;
}

.gr-textbox:focus-within, .gr-textbox textarea:focus, .gr-textbox input:focus,
textarea:focus, input[type="text"]:focus {
    border-color: var(--primary-color-light);
    outline: none;
    box-shadow: 0 0 0 2px rgba(0, 123, 255, 0.1);
}

.dark .gr-textbox, .dark .gr-textbox textarea, .dark .gr-textbox input,
.dark textarea, .dark input[type="text"] {
    background-color: #2d2d2d !important;
    color: var(--text-color-dark) !important;
    border-color: var(--border-color-dark) !important;
}

.dark .gr-textbox:focus-within, .dark .gr-textbox textarea:focus, .dark .gr-textbox input:focus,
.dark textarea:focus, .dark input[type="text"]:focus {
    border-color: var(--primary-color-dark) !important;
    box-shadow: 0 0 0 2px rgba(13, 110, 253, 0.1) !important;
}

/* File upload areas */
.gr-file, .gr-video, .gr-image {
    background-color: var(--bg-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 6px;
    font-family: 'Inter', sans-serif;
}

.dark .gr-file, .dark .gr-video, .dark .gr-image {
    background-color: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
    color: var(--text-color-dark) !important;
}

/* Labels */
.gr-label, label {
    color: var(--text-color-light);
    font-weight: 500;
    font-family: 'Inter', sans-serif;
    font-size: 14px;
    margin-bottom: 6px;
}

.dark .gr-label, .dark label {
    color: var(--text-color-dark) !important;
}

/* Cards and panels */
.gr-panel, .gr-box, .gr-form {
    background-color: var(--bg-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 8px;
    margin: 8px 0;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.05);
}

.dark .gr-panel, .dark .gr-box, .dark .gr-form {
    background-color: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.2) !important;
}

/* Buttons */
.gr-button {
    background: var(--primary-color-light);
    color: white;
    border: none;
    border-radius: 6px;
    padding: 10px 20px;
    font-weight: 500;
    font-family: 'Inter', sans-serif;
    font-size: 14px;
    transition: all 0.2s ease;
    cursor: pointer;
}

.gr-button:hover {
    background: #0056b3;
    transform: translateY(-1px);
    box-shadow: 0 2px 8px rgba(0, 123, 255, 0.2);
}

.dark .gr-button {
    background: var(--primary-color-dark);
}

.dark .gr-button:hover {
    background: #0b5ed7;
}

/* Tab navigation */
.tab-nav button {
    background: var(--primary-color-light);
    color: white;
    border: none;
    border-radius: 6px;
    padding: 10px 20px;
    margin: 2px;
    font-weight: 500;
    font-family: 'Inter', sans-serif;
    font-size: 14px;
    transition: all 0.2s ease;
}

.tab-nav button:hover {
    background: #0056b3;
}

.dark .tab-nav button {
    background: var(--primary-color-dark);
}

.dark .tab-nav button:hover {
    background: #0b5ed7;
}

/* Main header */
.main-header {
    text-align: center;
    font-size: 2.2em;
    font-weight: 600;
    margin-bottom: 24px;
    color: var(--primary-color-light);
    font-family: 'Inter', sans-serif;
    letter-spacing: -0.02em;
}

h1#main-title {
    font-family: 'Inter', sans-serif;
    font-weight: 600;
    font-size: 2.2rem;
    color: #1976d2;
    text-align: center;
    margin-bottom: 1rem;
    letter-spacing: -0.02em;
}

.dark .main-header {
    color: var(--primary-color-dark) !important;
}

/* Video section - full width */
.video-section {
    width: 100% !important;
    max-width: none !important;
}

.video-section .gr-video {
    width: 100% !important;
    min-height: 400px;
}

/* Results grid */
.results-grid {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 16px;
    margin-top: 16px;
}

.result-item {
    background: var(--bg-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 8px;
    padding: 16px;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.05);
}

.dark .result-item {
    background: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.2) !important;
}

/* Markdown content */
.gr-markdown {
    color: var(--text-color-light);
    font-family: 'Inter', sans-serif;
    line-height: 1.6;
}

.dark .gr-markdown {
    color: var(--text-color-dark) !important;
}

.dark .gr-markdown * {
    color: var(--text-color-dark) !important;
}

/* Image containers */
.image-container {
    display: flex;
    gap: 12px;
    justify-content: center;
    margin: 12px 0;
}

.image-item {
    flex: 1;
    max-width: 300px;
}

/* Sliders */
.gr-slider {
    color: var(--text-color-light);
    font-family: 'Inter', sans-serif;
}

.dark .gr-slider {
    color: var(--text-color-dark) !important;
}

/* Accordion */
.gr-accordion {
    background: var(--bg-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 6px;
    font-family: 'Inter', sans-serif;
}

.dark .gr-accordion {
    background: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
}

/* Cleanup styles */
.reduced-text {
    font-size: 14px;
    line-height: 1.5;
    font-family: 'Inter', sans-serif;
    color: var(--secondary-color-light);
}

.dark .reduced-text {
    color: var(--secondary-color-dark) !important;
}

.compact-section {
    margin: 8px 0;
    padding: 12px;
}

.chatbot {
    background: var(--bg-color-light);
    border: 1px solid var(--border-color-light);
    border-radius: 8px;
    font-family: 'Inter', sans-serif;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.05);
}

.dark .chatbot {
    background: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.2) !important;
}

/* Chat message styling */
.message {
    background: var(--bg-color-light);
    color: var(--text-color-light);
    border-radius: 6px;
    margin: 4px 0;
    font-family: 'Inter', sans-serif;
    font-size: 14px;
}

.dark .message {
    background: #404040 !important;
    color: var(--text-color-dark) !important;
}

/* User message */
.message.user {
    background: var(--primary-color-light);
    color: white;
}

.dark .message.user {
    background: var(--primary-color-dark) !important;
}

/* Bot message */
.message.bot {
    background: #f8f9fa;
    color: var(--text-color-light);
}

.dark .message.bot {
    background: #3d3d3d !important;
    color: var(--text-color-dark) !important;
}

/* Chat input area */
.chat-input-area {
    background: var(--bg-color-light);
    border-top: 1px solid var(--border-color-light);
    padding: 12px;
}

.dark .chat-input-area {
    background: #2d2d2d !important;
    border-color: var(--border-color-dark) !important;
}

/* Study tips panel */
.study-tips {
    background: rgba(0, 123, 255, 0.04);
    border: 1px solid rgba(0, 123, 255, 0.15);
    border-radius: 6px;
    padding: 12px;
    margin: 8px 0;
    font-family: 'Inter', sans-serif;
}

.dark .study-tips {
    background: rgba(13, 110, 253, 0.08) !important;
    border-color: rgba(13, 110, 253, 0.25) !important;
}

/* Enhanced spacing and typography */
h1, h2, h3, h4, h5, h6 {
    font-family: 'Inter', sans-serif;
    font-weight: 600;
    letter-spacing: -0.01em;
    margin-bottom: 0.5em;
}

p {
    font-family: 'Inter', sans-serif;
    line-height: 1.6;
    margin-bottom: 1em;
}

/* Subtle animations */
.gr-button, .tab-nav button {
    transition: all 0.2s cubic-bezier(0.4, 0, 0.2, 1);
}

.result-item, .gr-panel, .gr-box, .gr-form, .chatbot {
    transition: box-shadow 0.2s ease;
}

.result-item:hover, .gr-panel:hover, .gr-box:hover, .gr-form:hover {
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.08);
}

.dark .result-item:hover, .dark .gr-panel:hover, .dark .gr-box:hover, .dark .gr-form:hover {
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.3) !important;
}

.youtube-input-section {
    background: linear-gradient(45deg, #ff000015, #ff000008);
    border: 1px solid #ff000025;
    border-radius: 8px;
    padding: 16px;
    margin: 12px 0;
}

.dark .youtube-input-section {
    background: linear-gradient(45deg, #ff000020, #ff000010) !important;
    border-color: #ff000040 !important;
}

/* Input option sections */
.input-option-section {
    background: rgba(0, 123, 255, 0.02);
    border: 1px solid rgba(0, 123, 255, 0.1);
    border-radius: 6px;
    padding: 12px;
    margin: 8px 0;
}

.dark .input-option-section {
    background: rgba(13, 110, 253, 0.05) !important;
    border-color: rgba(13, 110, 253, 0.15) !important;
}

/* Section headers */
.section-header {
    color: var(--primary-color-light);
    font-weight: 600;
    font-size: 1.1em;
    margin-bottom: 8px;
    font-family: 'Inter', sans-serif;
}

.dark .section-header {
    color: var(--primary-color-dark) !important;
}

/* URL validation indicators */
.url-valid {
    border-color: #28a745 !important;
    box-shadow: 0 0 0 2px rgba(40, 167, 69, 0.1) !important;
}

.url-invalid {
    border-color: #dc3545 !important;
    box-shadow: 0 0 0 2px rgba(220, 53, 69, 0.1) !important;
}

/* Enhanced video section */
.video-upload-section {
    border: 2px dashed var(--border-color-light);
    border-radius: 8px;
    padding: 16px;
    text-align: center;
    transition: all 0.3s ease;
    background: rgba(0, 123, 255, 0.01);
}

.video-upload-section:hover {
    border-color: var(--primary-color-light);
    background: rgba(0, 123, 255, 0.03);
}

.dark .video-upload-section {
    border-color: var(--border-color-dark) !important;
    background: rgba(13, 110, 253, 0.03) !important;
}

.dark .video-upload-section:hover {
    border-color: var(--primary-color-dark) !important;
    background: rgba(13, 110, 253, 0.06) !important;
}

/* Processing status indicators */
.processing-status {
    background: rgba(0, 123, 255, 0.1);
    border: 1px solid rgba(0, 123, 255, 0.2);
    border-radius: 6px;
    padding: 8px 12px;
    margin: 4px 0;
    font-size: 14px;
    font-family: 'Inter', sans-serif;
    color: var(--primary-color-light);
}

.dark .processing-status {
    background: rgba(13, 110, 253, 0.15) !important;
    border-color: rgba(13, 110, 253, 0.3) !important;
    color: var(--primary-color-dark) !important;
}

/* Success/Error message styling */
.status-success {
    background: rgba(40, 167, 69, 0.1);
    border-color: rgba(40, 167, 69, 0.2);
    color: #155724;
}

.status-error {
    background: rgba(220, 53, 69, 0.1);
    border-color: rgba(220, 53, 69, 0.2);
    color: #721c24;
}

.dark .status-success {
    background: rgba(40, 167, 69, 0.15) !important;
    border-color: rgba(40, 167, 69, 0.3) !important;
    color: #28a745 !important;
}

.dark .status-error {
    background: rgba(220, 53, 69, 0.15) !important;
    border-color: rgba(220, 53, 69, 0.3) !important;
    color: #dc3545 !important;
}

/* Enhanced tip section */
.tip-section {
    background: linear-gradient(135deg, rgba(255, 193, 7, 0.1), rgba(255, 193, 7, 0.05));
    border: 1px solid rgba(255, 193, 7, 0.2);
    border-radius: 6px;
    padding: 10px 12px;
    margin: 8px 0;
    font-size: 13px;
    font-family: 'Inter', sans-serif;
}

.dark .tip-section {
    background: linear-gradient(135deg, rgba(255, 193, 7, 0.15), rgba(255, 193, 7, 0.08)) !important;
    border-color: rgba(255, 193, 7, 0.3) !important;
}

/* Input group styling */
.input-group {
    display: flex;
    flex-direction: column;
    gap: 12px;
    margin: 16px 0;
}

.input-group-header {
    font-weight: 500;
    color: var(--text-color-light);
    margin-bottom: 4px;
    font-family: 'Inter', sans-serif;
}

.dark .input-group-header {
    color: var(--text-color-dark) !important;
}

/* Progress bar enhancements */
.progress-container {
    background: rgba(0, 123, 255, 0.05);
    border-radius: 4px;
    padding: 8px;
    margin: 8px 0;
}

.dark .progress-container {
    background: rgba(13, 110, 253, 0.08) !important;
}

/* Enhanced button hover effects */
.gr-button:not(:disabled):hover {
    transform: translateY(-1px);
    box-shadow: 0 4px 12px rgba(0, 123, 255, 0.25);
}

.dark .gr-button:not(:disabled):hover {
    box-shadow: 0 4px 12px rgba(13, 110, 253, 0.3) !important;
}

/* Responsive adjustments */
@media (max-width: 768px) {
    .input-group {
        gap: 8px;
    }

    .video-upload-section {
        padding: 12px;
    }

    .section-header {
        font-size: 1em;
    }
}
"""

with gr.Blocks(css=custom_css, title="AI Classroom Assistant", theme=gr.themes.Soft()) as demo:
    gr.HTML('<h1 class="main-header">AI Classroom Assistant</h1>')

    with gr.Tab("Lecture Processor"):
        gr.Markdown(
            "Upload a lecture video or paste a YouTube URL to get transcript, notes, images, and resources",
            elem_classes=["reduced-text"]
        )

        with gr.Row():
            with gr.Column():
                gr.Markdown("### Option 1: Upload Video File")
                pipeline_video_input = gr.Video(
                    label="Upload Lecture Video",
                    height=300,
                    elem_classes=["video-section"]
                )

                gr.Markdown("### Option 2: YouTube URL")
                pipeline_youtube_input = gr.Textbox(
                    label="YouTube Video URL",
                    placeholder="https://www.youtube.com/watch?v=... or https://youtu.be/...",
                    lines=1,
                    elem_classes=["result-item"]
                )


        with gr.Row():
            with gr.Column(scale=1):
                with gr.Accordion("Settings", open=False):
                    pipeline_search_results = gr.Slider(5, 15, value=8, label="Search Results")
                    pipeline_num_images = gr.Slider(1, 4, value=2, label="Images")
                pipeline_btn = gr.Button("Process Lecture", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("", elem_classes=["reduced-text"])

        with gr.Row():
            with gr.Column():
                pipeline_transcript = gr.Textbox(
                    label="Transcript",
                    lines=8,
                    show_copy_button=True,
                    elem_classes=["result-item"]
                )
                pipeline_notes = gr.Textbox(
                    label="Structured Notes",
                    lines=8,
                    show_copy_button=True,
                    elem_classes=["result-item"]
                )

            with gr.Column():
                with gr.Row():
                    pipeline_image1 = gr.Image(label="Related Image 1", height=180)
                    pipeline_image2 = gr.Image(label="Related Image 2", height=180)
                pipeline_search = gr.Textbox(
                    label="Web Resources",
                    lines=8,
                    show_copy_button=True,
                    elem_classes=["result-item"]
                )

        pipeline_btn.click(
            process_youtube_or_upload_robust,
            inputs=[
                pipeline_video_input,
                pipeline_youtube_input,
                pipeline_search_results,
                pipeline_num_images
            ],
            outputs=[
                pipeline_transcript,
                pipeline_notes,
                pipeline_image1,
                pipeline_image2,
                pipeline_search
            ],
            show_progress=True
        )

    with gr.Tab("Transcription"):
        gr.Markdown("Convert video speech to text", elem_classes=["reduced-text"])
        with gr.Row():
            with gr.Column():
                video_input = gr.Video(label="Upload Video", height=250)
                trans_btn = gr.Button("Transcribe", variant="primary")
            with gr.Column():
                transcription_out = gr.Textbox(label="Transcript", lines=10, show_copy_button=True)
        trans_btn.click(transcribe_audio_from_video, inputs=video_input, outputs=transcription_out, show_progress=True)



    with gr.Tab("Notes"):
        gr.Markdown("Generate structured notes from text", elem_classes=["reduced-text"])
        with gr.Row():
            with gr.Column():
                notes_transcript = gr.Textbox(label="Input Text", lines=8)
                notes_btn = gr.Button("Generate Notes", variant="primary")
            with gr.Column():
                notes_output = gr.Textbox(label="Structured Notes", lines=10, show_copy_button=True)

        notes_btn.click(generate_structured_notes, inputs=[notes_transcript], outputs=notes_output, show_progress=True)

    with gr.Tab("Search"):
        gr.Markdown("Search web and videos", elem_classes=["reduced-text"])
        with gr.Row():
            with gr.Column():
                search_query = gr.Textbox(label="Search Query")
                with gr.Row():
                    search_results_num = gr.Slider(3, 15, value=8, label="Results")
                    include_videos_check = gr.Checkbox(label="Include Videos", value=True)
                search_btn = gr.Button("Search", variant="primary")
            with gr.Column():
                search_output = gr.Textbox(label="Results", lines=10, show_copy_button=True)

        search_btn.click(
            search_web_enhanced,
            inputs=[search_query, search_results_num, include_videos_check],
            outputs=search_output,
            show_progress=True
        )

    with gr.Tab("Images"):
        gr.Markdown("Search for relevant images", elem_classes=["reduced-text"])
        with gr.Row():
            with gr.Column():
                image_search_query = gr.Textbox(label="Image Search Query")
                image_num_slider = gr.Slider(1, 6, value=2, label="Number of Images")
                image_search_btn = gr.Button("Search Images", variant="primary")
            with gr.Column():
                with gr.Row():
                    image_output1 = gr.Image(label="Image 1", height=200)
                    image_output2 = gr.Image(label="Image 2", height=200)
                with gr.Row():
                    image_output3 = gr.Image(label="Image 3", height=200)
                    image_output4 = gr.Image(label="Image 4", height=200)

        def search_and_display_images(query, num_images):
            if not query.strip():
                return [None] * 4

            image_urls = search_google_images(query, min(num_images, 4))
            images = []
            for url in image_urls:
                img = download_image_from_url(url)
                images.append(img)

            while len(images) < 4:
                images.append(None)

            return images[:4]

        image_search_btn.click(
            search_and_display_images,
            inputs=[image_search_query, image_num_slider],
            outputs=[image_output1, image_output2, image_output3, image_output4],
            show_progress=True
        )

    with gr.Tab("Study Chat"):
        gr.Markdown("Ask questions about your studies", elem_classes=["reduced-text"])

        with gr.Row():
            with gr.Column(scale=4):
                chatbot_interface = gr.Chatbot(
                    label="AI Study Assistant",
                    height=500,
                    show_copy_button=True,
                    bubble_full_width=False,
                    elem_classes=["result-item"]
                )

                with gr.Row():
                    with gr.Column(scale=4):
                        chat_input = gr.Textbox(
                            label="Ask a question",
                            placeholder="Ask me anything about your studies - concepts, definitions, explanations...",
                            lines=2,
                            max_lines=4
                        )
                    with gr.Column(scale=1, min_width=100):
                        chat_send_btn = gr.Button("Send", variant="primary", size="sm")

                with gr.Row():
                    clear_chat_btn = gr.Button("Clear Chat", variant="secondary", size="sm")

        def handle_chat_submit(message, history):
            return generate_chatbot_response(message, history), ""

        chat_input.submit(
            handle_chat_submit,
            inputs=[chat_input, chatbot_interface],
            outputs=[chatbot_interface, chat_input]
        )

        chat_send_btn.click(
            handle_chat_submit,
            inputs=[chat_input, chatbot_interface],
            outputs=[chatbot_interface, chat_input]
        )

        clear_chat_btn.click(
            clear_chat_history,
            outputs=[chatbot_interface]
        )

    gr.HTML('<div style="text-align: center; margin-top: 30px; padding: 20px; background: rgba(0,123,255,0.1); border-radius: 10px; font-family: Inter, sans-serif;"><h3>Made by Kewal Thacker and Siddharth Subramanian</h3></div>')

demo.launch(share=True, debug=True)

  chatbot_interface = gr.Chatbot(



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://71588b56766273e9ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Loading chatbot model...
🔄 Converting chatbot model to OpenVINO...


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
  or not self.key_cache[layer_idx].numel()  # the layer has no cache

  if sequence_length != 1:

  elif (

  is_causal = query.shape[2] > 1 and causal_mask is None



✅ Chatbot model loaded successfully!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://71588b56766273e9ea.gradio.live
